# 問題
26の処理に加えて、テンプレートの値からMediaWikiの内部リンクマークアップを除去し、テキストに変換せよ

In [6]:
import pandas as pd
import re

df = pd.read_json("Jawiki Country.json.gz", lines=True)
uk_text = df.query('title == "イギリス"')["text"].values[0]

# 1) 基礎情報ブロックを抜く（終端の空白も許容）
block_pat = r'^\{\{\s*基礎情報[^\n]*.*?^\}\}\s*$'
blocks = re.findall(block_pat, uk_text, flags=re.DOTALL | re.MULTILINE)
if not blocks:
    raise ValueError("基礎情報ブロックが見つかりません")
block = blocks[0]

# 2) フィールド抽出：次のフィールド行 or 終端 or 文字列末まで
field_pat = r'^\|\s*(.+?)\s*=\s*(.*?)(?=\n\||\n\}\}|$)'

# 3) 強調マークアップ除去関数（抽出後に適用）
def remove_emphasis(text):
    patterns = [
        (r"'''''(.*?)'''''", r"\1"),  # 強い強調(pat, repl)
        (r"'''(.*?)'''",     r"\1"),  # 強調
        (r"''(.*?)''",       r"\1"),  # 弱い強調
    ]
    changed = True
    while changed:
        old = text
        for pat, repl in patterns:
            text = re.sub(pat, repl, text, flags=re.DOTALL)
        changed = (text != old)
    return text.strip()

def remove_linkmark(text):
    patterns = [
        # 1) メディア系（File/ファイル/画像）: 最後の説明テキストを優先
        (r"\[\[(?:ファイル|File|画像):[^|\]]+(?:\|[^|\]]+)*\|([^|\]=][^|\]]*)\]\]", r"\1", 0),
        #    説明が無い純粋なメディアリンクは、ファイル名（接頭辞後）を残す
        (r"\[\](?!)", r"", 0),  # ダミー行（編集しやすさ用・実質無効）
        (r"\[\[(?:ファイル|File|画像):([^\]|]+)\]\]", r"\1", 0),

        # 2) 内部リンク：[[記事|表示]] → 表示
        (r"\[\[[^|\]]+\|([^\]]+)\]\]", r"\1", 0),

        # 3) 内部リンク：[[記事]] / [[記事#節]] → 記事
        (r"\[\[([^\]|#]+)(?:#[^\]]+)?\]\]", r"\1", 0),

        # 4) インターウィキ：[[:en:Title|表示]] → 表示
        (r"\[\[:[a-z0-9\-]+:[^|\]]+\|([^\]]+)\]\]", r"\1", re.I),
        #    インターウィキ：[[:en:Title]] → Title（#節は落とす）
        (r"\[\[:[a-z0-9\-]+:([^\]#|]+)(?:#[^\]]+)?\]\]", r"\1", re.I),
    ]

    changed = True
    while changed:
        old = text
        for pat, repl, flags in patterns:
            text = re.sub(pat, repl, text, flags=flags)
            changed = (text != old)
    return text.strip()

# 4) 辞書化（組み込み名を上書きしない）
info = {}
for m in re.finditer(field_pat, block, flags=re.DOTALL | re.MULTILINE):
    name = m.group(1).strip()
    value = m.group(2).strip()
    value = remove_emphasis(value)  
    value = remove_linkmark(value)
    info[name] = value

info

{'略名': 'イギリス',
 '日本語国名': 'グレートブリテン及び北アイルランド連合王国',
 '公式国名': '{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />',
 '国旗画像': 'Flag of the United Kingdom.svg',
 '国章画像': 'イギリスの国章',
 '国章リンク': '（国章）',
 '標語': '{{lang|fr|Dieu et mon droit}}<br />（フランス語:神と我が権利）',
 '国歌': '{{lang|en|God Save the Queen}}{{en icon}}<br />神よ女王を護り賜え<br />{{center|United States Navy Band - God Save the Queen.ogg}}',
 '地図画像': 'Europe-UK.svg',
 '位置画像': 'United Kingdom (+overseas territories) in the World (+Antarctica claims).svg',
 '公用語': '英語',
 '首都': 'ロンドン（事実上）',
 '最大都市': 'ロンドン',
 '元首等肩書': '女王',
 '元首等氏名': 'エリザベス2世',
 '首相等肩書': '首相',
 '首相等氏名': 'ボリス・ジョンソン',
 '他元首等肩書1': '貴族院議長',
 '他元首等氏名1': 'ノーマン・ファウラー',
 '他元首等肩書2': '庶民院議長',
 '他元首等氏名2': '{{仮リンク|リンゼイ・ホイル|en|Lindsay Hoyle}}',
 '他元首等肩書3': '最高裁判所長官',
 '他元首等氏名3': 'ブレンダ・ヘイル',
 '面積順位': '76',
 '面積大きさ': '1 E11',
 '面積値': '244,820',
 '水面積率': '1.3%',
 '人口統計年': '2018',
 '人口順位': '22',
 '人口大きさ': '1 E7',
 '人口値': '6643万5600<ref>{{Cite web|url=https://www.

# remove_linkmark 関数の解説
Wikipedia のマークアップを普通のテキストに変換する関数です。
⸻

## 📌 全体の流れ
	1.	patterns に「探すパターン・置換後文字列・フラグ」をまとめておく
	2.	re.sub で順番に置換していく
	3.	変化がなくなるまで繰り返す（入れ子のケースにも対応）
	4.	最後に 余計な空白を削除

⸻

## 📌 patterns の構造
patterns = [
    (正規表現パターン, 置換後文字列, フラグ),
    ...
]

## 📌 各パターンの意味
1) \{\{lang\|[a-z\-]+\|([^}]+)\}\}
- \{\{ … {{ のリテラル。テンプレート開始。
- lang\| … 「lang|」の文字列。言語指定テンプレを狙う。
- [a-z\-]+ … 小文字アルファベットとハイフンの並び。言語コード（en, zh-cn など）。
- \| … 区切りのパイプ。
- ([^}]+) … キャプチャ。} 以外を1文字以上。つまり本文テキスト。
- \}\} … }} のリテラル。テンプレート終了。
👉 目的: {{lang|en|United Kingdom}} の中から「United Kingdom」だけを取り出す。

⸻

2) \{\{仮リンク\|([^|}]+)\|[^}]+\}\}
-	\{\{仮リンク\| … 「{{仮リンク|」に一致。
-	([^|}]+) … キャプチャ。パイプや閉じ波括弧でない部分 → 表示テキスト。
-	\|[^}]+ … その後の「言語コードや英語名」をスキップ。
-	\}\} … }} で終わり。
👉 目的: {{仮リンク|リンゼイ・ホイル|en|Lindsay Hoyle}} の中から「リンゼイ・ホイル」だけを取り出す。

⸻

3) \[\[(?:ファイル|File|画像):[^|\]]+(?:\|[^|\]]+)*\|([^|\]=][^|\]]*)\]\]
-	\[\ …  の開始。
-	(?:ファイル|File|画像): … 「ファイル:」「File:」「画像:」のいずれかに一致。非キャプチャグループ。
-	[^|\]]+ … パイプや  以外を1文字以上。ファイル名部分。
-	(?:\|[^|\]]+)* … 任意の数の追加パラメータ（例: |thumb|85px）。
-	\|([^|\]=][^|\]*) … キャプチャ。最後の「説明文」だけを取る。ただし先頭が = なら除外（alt=... などを避ける工夫）。
-	\\] … ]] の終わり。
👉 目的: [[File:Flag.svg|thumb|国旗]] から「国旗」だけを取り出す。

⸻

4) \[\[[^|\]]+\|([^\]]+)\]\]
-	\[\[ … [[ 開始。
-	[^|\]]+ … パイプや ] 以外を全部（リンク先）。
-	\|([^\]]+) … パイプの後、閉じ ] までをキャプチャ（表示テキスト）。
-	\]\] … 閉じ。
👉 目的: [[ロンドン|首都]] → 「首都」を取り出す。

⸻

5) \[\[([^\]|#]+)(?:#[^\]]+)?\]\]
-	\[\[ … 開始。
-	([^\]|#]+) … キャプチャ。| や ] や # 以外を1文字以上 → 記事名。
-	(?:#[^\]]+)? … オプションで #節名 を許容（ただしキャプチャしない）。
-	\]\] … 終了。
👉 目的: [[化学#歴史]] → 「化学」、[[ロンドン]] → 「ロンドン」。

⸻

6) \[\[:[a-z\-]+:[^|\]]+\|([^\]]+)\]\]
-	\[\ … 開始。
-	: … 冒頭コロン。インターウィキやカテゴリ等を普通のリンクとして書く形式。
-	[a-z\-]+: … 言語コードやプロジェクト名。en: など。
-	[^|\]]+ … パイプまたは  までを記事名として読み飛ばす。
-	\|([^\]]+) … パイプの右、表示テキストをキャプチャ。
-	\]\] … 閉じ。
👉 目的: [[:en:United Kingdom|イギリス]] → 「イギリス」。

⸻

7) \[\[:[a-z\-]+:([^\]]+)\]\]
-	\[\[ … 開始。
-	: … 冒頭コロン。
-	[a-z\-]+: … インターウィキ接頭辞（例: en:）。
-	([^\]]+) … キャプチャ。閉じ ] までの本文全部。
-	\]\] … 終了。
👉 目的: [[:en:United Kingdom]] → 「United Kingdom」。

⸻

8) <ref[^>]*>.*?</ref>
-	<ref … ref タグの開始。
-	[^>]* … 属性（name=…など）を無視。
-	> … 開きタグの終端。
-	.*? … 最短一致で中身。
-	</ref> … 閉じタグ。
👉 目的: 脚注本文ごと削除する。

⸻

9) <references\s*/>
-	<references … references タグの開始。
-	\s* … 空白を許可。
-	/ > … 閉じ。
👉 目的: 脚注展開タグを削除。

⸻

10) <br\s*/?>
-	<br … br タグ開始。
-	\s* … 空白を許可。
-	/?> … / があってもなくても閉じにマッチ。
👉 目的: <br> または <br /> を改行に変換。

⸻

11) \{\{[^}]+\}\}
-	\{\{ … {{ 開始。
-	[^}]+ … } 以外を貪欲に。中身全部。
-	\}\} … }} 終了。
👉 目的: その他のテンプレを一括削除（例: {{en icon}} など）。

⸻

💡 まとめると：
-	{{…}} 系 → テンプレの中身を抜き出すか削除する
-	[[…]] 系 → 内部リンクやインターウィキの「表示テキスト」だけを残す
-	<…> 系 → HTMLタグを削除または改行に変換

⸻

